2.1 Preparing MNIST math dataset

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

( train_ds , test_ds ) , ds_info = tfds . load (’mnist ’, split =[ ’train ’, ’
test ’], as_supervised = True , with_info = True )

# should we consider using .ipynb than .py? I

2.2 Two MNIST math datasets

In [ ]:
def prepare_data(dataset):

    '''
    :param dataset: the dataset to be prepared for input into the network
    :return: dataset run through the prep steps below
    '''

    # first step
    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # flatten the images into vectors
    dataset = dataset.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))
    # create one-hot targets
    dataset = dataset.map(lambda img, target: (img, tf.one_hot(target, depth=10)))

    # second step (this is an example from flipped classroom04, we can configure it to fit hw4)
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((dataset.shuffle(2000), 
                                     dataset.shuffle(2000)))

    # map ((x1,y1),(x2,y2)) to (x1,x2, y1==y2*) *boolean
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1]==x2[1]))
    # transform boolean target to int
    zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))

    #third step
    # shuffle, batch, prefetch
    zipped_ds = zipped_ds.shuffle(1000)
    zipped_ds = zipped_ds.batch(2**6)
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    
    return zipped_ds